1. Фреймфорк: torchvision.models.detection
3. Датасет: WIDER face

Я не сог обучить модель. Не понимаю, что делать с переполнением памяти в CUDA, в google colab.

In [1]:
!gdown --id 1ak1rZgRDfxXYIFg7RUx8o8bTLwHntdee
!unzip WIDER.zip > /dev/null

Downloading...
From: https://drive.google.com/uc?id=1ak1rZgRDfxXYIFg7RUx8o8bTLwHntdee
To: /content/WIDER.zip
100% 3.68G/3.68G [00:30<00:00, 120MB/s]


In [2]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 91740, done.
remote: Counting objects: 100% (26871/26871), done.
remote: Compressing objects: 100% (2367/2367), done.
remote: Total 91740 (delta 24733), reused 26284 (delta 24364), pack-reused 64869
Receiving objects: 100% (91740/91740), 181.56 MiB | 27.09 MiB/s, done.
Resolving deltas: 100% (77808/77808), done.
Note: checking out 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 2f40a483d [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [31]:
import os
from os.path import abspath, expanduser
from typing import Any, Callable, List, Dict, Optional, Tuple, Union
import numpy as np
import torch
import torch.utils.data
from PIL import Image

from torchvision.datasets.utils import (
    download_file_from_google_drive,
    download_and_extract_archive,
    extract_archive,
    verify_str_arg,
)
from torchvision.datasets.vision import VisionDataset

class WiderDataset(VisionDataset):
    BASE_FOLDER = "WIDER"
    FILE_LIST = [
        # File ID                             MD5 Hash                            Filename
        ("15hGDLhsx8bLgLcIRD5DhYt5iBxnjNF1M", "3fedf70df600953d25982bcd13d91ba2", "WIDER_train.zip"),
        ("1GUCogbp16PMGa39thoMMeWxp7Rp5oM8Q", "dfa7d7e790efa35df3788964cf0bbaea", "WIDER_val.zip"),
        ("1HIfDbVEWKmsYKJZm4lchTBDLW5N7dY5T", "e5d8f4248ed24c334bbd12f49c29dd40", "WIDER_test.zip"),
    ]
    ANNOTATIONS_FILE = (
        "http://shuoyang1213.me/WIDERFACE/support/bbx_annotation/wider_face_split.zip",
        "0e3767bcf0e326556d407bf5bff5d27c",
        "wider_face_split.zip",
    )
    def __init__(
        self,
        root: str,
        split: str = "train",
        transform: Optional[Callable] = None,
        download: bool = False,
    ) -> None:
        super().__init__(root=os.path.join(root, self.BASE_FOLDER), transform=transform)
        
        # check arguments
        self.split = verify_str_arg(split, "split", ("train", "val", "test"))

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError("Dataset not found or corrupted. You can use download=True to download and prepare it")

        self.img_info: List[Dict[str, Union[str, Dict[str, torch.Tensor]]]] = []
        if self.split in ("train", "val"):
            self.parse_train_val_annotations_file()
        else:
            self.parse_test_annotations_file()

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        
        img = Image.open(self.img_info[index]["img_path"]).convert("RGB")
        target = None if self.split == "test" else self.img_info[index]["annotations"]
        if self.transform is not None:
            img, target = self.transform(img, target)

        return img, target

    def __len__(self) -> int:
        return len(self.img_info)
    
    def extra_repr(self) -> str:
        lines = ["Split: {split}"]
        return "\n".join(lines).format(**self.__dict__)

    def parse_train_val_annotations_file(self) -> None:
        filename = "wider_face_train_bbx_gt.txt" if self.split == "train" else "wider_face_val_bbx_gt.txt"
        filepath = os.path.join(self.root, "wider_face_split", filename)

        with open(filepath) as f:
            lines = f.readlines()
            file_name_line, num_boxes_line, box_annotation_line = True, False, False
            num_boxes, box_counter, k = 0, 0, 0
            labels = []
            for line in lines:
                line = line.rstrip()
                if file_name_line: 
                    img_path = os.path.join(self.root, "WIDER_" + self.split, "images", line)
                    img_path = abspath(expanduser(img_path))
                    file_name_line = False
                    num_boxes_line = True
                elif num_boxes_line:
                    num_boxes = int(line)
                    num_boxes_line = False
                    box_annotation_line = True
                elif box_annotation_line:
                    box_counter += 1
                    line_split = line.split(" ")
                    line_values = [int(x) for x in line_split]
                    line_values = np.array(line_values, dtype=np.float32)

                    xmin = line_values[0].copy()
                    xmax = line_values[2].copy()
                    ymin = line_values[1].copy()
                    ymax = line_values[3].copy()

                    if xmin > xmax:
                        xmin, xmax = xmax, xmin
                    elif xmin == xmax:
                        xmax += 1

                    if ymin > ymax:
                        ymin, ymax = ymax, ymin
                    elif ymin == xmax:
                        ymax += 1

                    line_values = []
                    line_values = np.array(line_values, dtype=np.float32)
                    line_values = np.append(line_values, xmin)
                    line_values = np.append(line_values, ymin)
                    line_values = np.append(line_values, xmax)
                    line_values = np.append(line_values, ymax)
                    if xmin < xmax and ymin < ymax and ((xmax - xmin)*(ymax-ymin)) > 0:
                        labels.append(line_values)
                    else:
                        k += 1
                    
                    if box_counter >= num_boxes:
                        box_annotation_line = False
                        file_name_line = True
                        if num_boxes-k > 0:
                            labels_tensor = torch.tensor(labels)
                            area = (labels_tensor[:, 3] - labels_tensor[:, 1])*(labels_tensor[:, 2] - labels_tensor[:, 0])
                            self.img_info.append(
                                { 
                                    "img_path": img_path,
                                    "annotations": {
                                        "boxes": labels_tensor[:, 0:4],
                                        "labels": torch.ones((box_counter-k,), dtype=torch.int64),
                                        "image_id": torch.ones(1),
                                        "area": area,
                                        "iscrowd": torch.zeros((box_counter-k,), dtype=torch.int64)
                                    }
                                }
                            )
                        box_counter, k = 0, 0
                        labels.clear()
                else:
                    raise RuntimeError(f"Error parsing annotation file {filepath}")

    def parse_test_annotations_file(self) -> None:
        filepath = os.path.join(self.root, "wider_face_split", "wider_face_test_filelist.txt")
        filepath = abspath(expanduser(filepath))
        with open(filepath) as f:
            lines = f.readlines()
            for line in lines:
                line = line.rstrip()
                img_path = os.path.join(self.root, "WISER_test", "images", line)
                img_path = abspath(expanduser(img_path))
                self.img_info.append({"img_path": img_path})

    def _check_integrity(self) -> bool:
        # Allow original archive to be deleted (zip). Only need the extracted images
        all_files = self.FILE_LIST.copy()
        all_files.append(self.ANNOTATIONS_FILE)
        for (_, md5, filename) in all_files:
            file, ext = os.path.splitext(filename)
            extracted_dir = os.path.join(self.root, file)
            if not os.path.exists(extracted_dir):
                return False
        return True

    def download(self) -> None:
        if self._check_integrity():
            print("Files already downloaded and verified")
            return

        # download and extract image data
        for (file_id, md5, filename) in self.FILE_LIST:
            download_file_from_google_drive(file_id, self.root, filename, md5)
            filepath = os.path.join(self.root, filename)
            extract_archive(filepath)

        # download and extract annotation files
        download_and_extract_archive(
            url=self.ANNOTATIONS_FILE[0], download_root=self.root, md5=self.ANNOTATIONS_FILE[1]
        )

In [22]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [32]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork
import torch

fasterRCNN = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# Define RPN 
anchor_generator = AnchorGenerator(sizes=tuple([(16, 32, 64, 128, 256) for _ in range(5)]), # let num of tuple equal to num of feature maps
                                  aspect_ratios=tuple([(0.75, 0.5, 1.25) for _ in range(5)])) # ref: https://github.com/pytorch/vision/issues/978
rpn_head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
fasterRCNN.rpn = RegionProposalNetwork(
    anchor_generator= anchor_generator, head= rpn_head,
    fg_iou_thresh= 0.7, bg_iou_thresh=0.3,
    batch_size_per_image=48, # use fewer proposals
    positive_fraction = 0.5,
    pre_nms_top_n=dict(training=200, testing=100),
    post_nms_top_n=dict(training=160, testing=80),
    nms_thresh = 0.7
)

In [33]:
in_features = fasterRCNN.roi_heads.box_predictor.cls_score.in_features #get number of features
fasterRCNN.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes = 2)
fasterRCNN.roi_heads.fg_bg_sampler.batch_size_per_image = 24
fasterRCNN.roi_heads.fg_bg_sampler.positive_fraction = 0.5

In [34]:
# use our dataset and defined transformations
dataset = WiderDataset('WIDER', 'train',  get_transform(train=True))
dataset_test = WiderDataset('WIDER', 'val',  get_transform(train=True))

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [35]:
# move model to the right device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
fasterRCNN.to(device)

params = [p for p in fasterRCNN.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0005, betas=(0.9, 0.999), weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
metric_collector = []
num_epochs = 15
for epoch in range(num_epochs):
    # train for one epoch, printing every 5 iterations
    metric_logger = train_one_epoch(fasterRCNN, optimizer, data_loader, device, epoch, print_freq=5)
    metric_collector.append(metric_logger)
    # update the learning rate
    lr_scheduler.step()
    # Evaluate with validation dataset
    metric_logger_val = validate(fasterRCNN, data_loader_test, device, print_freq=5)
    #save checlpoint
    torch.save( fasterRCNN.state_dict(), os.path.join( weights_path,'fasterRCNN_ep'+str(epoch)+'.pth') )


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: [0]  [   0/6425]  eta: 4:00:24  lr: 0.000001  loss: 2.6776 (2.6776)  loss_classifier: 0.8727 (0.8727)  loss_box_reg: 0.0806 (0.0806)  loss_objectness: 0.6794 (0.6794)  loss_rpn_box_reg: 1.0449 (1.0449)  time: 2.2451  data: 0.7530  max mem: 10805
Epoch: [0]  [   5/6425]  eta: 2:57:17  lr: 0.000003  loss: 2.5917 (2.4297)  loss_classifier: 0.8664 (0.8491)  loss_box_reg: 0.1662 (0.2299)  loss_objectness: 0.6794 (0.6791)  loss_rpn_box_reg: 0.7693 (0.6716)  time: 1.6570  data: 0.1347  max mem: 10805


RuntimeError: ignored